In [ ]:
import requests
import pandas as pd
import getpass

In [ ]:
#Input Data - Login Details, game week required, league id required
email = getpass.getpass('Login_email:')
pswd = getpass.getpass('Password:')
gw = int(getpass.getpass('Gameweek:'))
league_id = getpass.getpass('League_id:')

In [ ]:
#Login into the FPL API and get League details
payload = {"login": email, "password": pswd,
           "app": "plfpl-web", "redirect_uri": "https://fantasy.premierleague.com/"}

with requests.Session() as session:
    session.post("https://users.premierleague.com/accounts/login/", data=payload)
    league_json = session.get("https://fantasy.premierleague.com/api/leagues-classic/"+str(league_id)+"/standings/").json()


league_entries = []
league_player_names = []

for entry in range(len(league_json["standings"]["results"])):
    league_entries.append(league_json["standings"]["results"][entry]["entry"])
    league_player_names.append(league_json["standings"]["results"][entry]["player_name"])
    
print('The league has ' + str(len(league_entries)) + ' Players')
print(*league_player_names, sep = "\n")


In [ ]:
#Position for a given element - Basically player id - position id mapping
with requests.Session() as session:
        session.post("https://users.premierleague.com/accounts/login/", data=payload)
        json_return_all_boot = session.get("https://fantasy.premierleague.com/api/bootstrap-static/").json()


element_df = pd.DataFrame(json_return_all_boot["elements"])[["id","element_type"]]


#Get the points of each player id - for the given gameweek
with requests.Session() as session:
        session.post("https://users.premierleague.com/accounts/login/", data=payload)
        json_return_live = session.get("https://fantasy.premierleague.com/api/event/" + str(gw) + "/live/").json()

point_df = pd.DataFrame(columns=["id","total_points"])
for x in range(len(json_return_live["elements"])):
    player_id = json_return_live["elements"][x]["id"]
    pts = json_return_live["elements"][x]["stats"]["total_points"]
    point_df = point_df.append(pd.DataFrame(zip([player_id],[pts]),columns=["id","total_points"])).reset_index(drop=True)

player_point_df = pd.merge(element_df,point_df,how='left',on=['id'])

print('The points scored by each player in the required FPL Gameweek has been fetched')

In [ ]:
#Creating an empty dataframe to append to 
final_df = pd.DataFrame()

#Looping through the league entries
i=0
for player in league_entries:
    
    i=i+1
    
    with requests.Session() as session:
        session.post("https://users.premierleague.com/accounts/login/", data=payload)
        json_return = session.get("https://fantasy.premierleague.com/api/entry/"+str(player)+"/history/").json()
    
    dict_val = pd.DataFrame(json_return["current"])
    play_df = dict_val.loc[dict_val['event']==gw].reset_index(drop=True)
    play_df['player_id'] = player
    play_df['player_name'] = league_player_names[i-1]
    play_df['gameweek'] = play_df['event'] 
    play_df['gw_points'] = play_df['points'] - play_df['event_transfers_cost']
    play_df['value'] = play_df['value']/10
    play_df['bank'] = play_df['bank']/10
    
    play_df_new = play_df[['player_name','player_id','gameweek','gw_points','points_on_bench'
                           ,'event_transfers', 'value']].reset_index(drop=True)
    
    #Gives Team List in the form of elements - 
    link = 'https://fantasy.premierleague.com/api/entry/' + str(player) + '/event/' + str(gw) + '/picks/'
    
    with requests.Session() as session:
            session.post("https://users.premierleague.com/accounts/login/", data=payload)
            json_return_new = session.get(link).json()
            
    df_picks = pd.DataFrame(json_return_new["picks"])
    
    df_picks_new = pd.merge(df_picks,player_point_df,how='left',left_on='element', right_on='id')
    
    df_pos_agg = df_picks_new[df_picks_new['position']<=11][['element_type','total_points']].groupby(['element_type']).sum().reset_index()
    
    pos_dict ={1:'def_points',2:'def_points',3:'mid_points',4:'fwd_points'}
    
    df_pos_agg = df_pos_agg.replace({'element_type':pos_dict})
    df_pos_agg = df_pos_agg.groupby(['element_type']).sum().reset_index()
    df_pos_agg = pd.pivot_table(df_pos_agg,columns = 'element_type',values = 'total_points').reset_index(drop=True)[['def_points','mid_points','fwd_points']]
    df_pos_agg.columns.name= None
    
    first_df = pd.concat([play_df_new,df_pos_agg],axis=1)
    
    final_df = final_df.append(first_df)
    print(league_player_names[i-1] + ' Completed')

final_df = final_df.sort_values(['player_name']).reset_index(drop=True)

final_df

In [ ]:
final_df.to_csv(path_or_buf = 'Gameweek '+str(gw)+'.csv', index=False)
print('The gameweek csv file is exported to your home location')